In [26]:
import tensorflow as tf

In [27]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

n_input = 784
n_classes = 10
dropout = 0.75

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [29]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [30]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                           padding='SAME')

In [31]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out

In [35]:
weights = {
    'wc1':tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2':tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1':tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out':tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred = conv_net(x, weights, biases, keep_prob)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1})
        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], 
                                 feed_dict={x:batch_xs, y:batch_ys, keep_prob:1})
            print("Iter " + str(step * batch_size) + ", Minbatch Loss=" + \
                 "{:.6f}".format(loss) + ",Training Accuracy=" + \
                 "{:.5f}".format(acc))
        
        step += 1
        
    print("Optimization Finished!")
    
    print("Testing Accuracy:", \
         sess.run(accuracy, feed_dict={x:mnist.test.images[:256],
                                      y:mnist.test.labels[:256],
                                      keep_prob:1}))

Iter 1280, Minbatch Loss=21926.994141,Training Accuracy=0.33594
Iter 2560, Minbatch Loss=12424.623047,Training Accuracy=0.46094
Iter 3840, Minbatch Loss=4379.718262,Training Accuracy=0.72656
Iter 5120, Minbatch Loss=4041.018555,Training Accuracy=0.74219
Iter 6400, Minbatch Loss=3645.779297,Training Accuracy=0.79688
Iter 7680, Minbatch Loss=3589.549316,Training Accuracy=0.75781
Iter 8960, Minbatch Loss=2413.432129,Training Accuracy=0.84375
Iter 10240, Minbatch Loss=2576.201416,Training Accuracy=0.78125
Iter 11520, Minbatch Loss=2500.846436,Training Accuracy=0.82031
Iter 12800, Minbatch Loss=2786.000244,Training Accuracy=0.84375
Iter 14080, Minbatch Loss=1152.484741,Training Accuracy=0.88281
Iter 15360, Minbatch Loss=1883.857666,Training Accuracy=0.84375
Iter 16640, Minbatch Loss=3404.389404,Training Accuracy=0.85938
Iter 17920, Minbatch Loss=2180.581787,Training Accuracy=0.82812
Iter 19200, Minbatch Loss=783.664734,Training Accuracy=0.89062
Iter 20480, Minbatch Loss=1501.786987,Training

Iter 166400, Minbatch Loss=233.168793,Training Accuracy=0.96875
Iter 167680, Minbatch Loss=77.139175,Training Accuracy=0.99219
Iter 168960, Minbatch Loss=85.896713,Training Accuracy=0.98438
Iter 170240, Minbatch Loss=41.519623,Training Accuracy=0.98438
Iter 171520, Minbatch Loss=473.255676,Training Accuracy=0.96875
Iter 172800, Minbatch Loss=51.886871,Training Accuracy=0.98438
Iter 174080, Minbatch Loss=70.299088,Training Accuracy=0.97656
Iter 175360, Minbatch Loss=34.731049,Training Accuracy=0.99219
Iter 176640, Minbatch Loss=135.372635,Training Accuracy=0.99219
Iter 177920, Minbatch Loss=104.242439,Training Accuracy=0.96875
Iter 179200, Minbatch Loss=70.726700,Training Accuracy=0.98438
Iter 180480, Minbatch Loss=278.204071,Training Accuracy=0.96875
Iter 181760, Minbatch Loss=0.000000,Training Accuracy=1.00000
Iter 183040, Minbatch Loss=388.502625,Training Accuracy=0.95312
Iter 184320, Minbatch Loss=511.827057,Training Accuracy=0.94531
Iter 185600, Minbatch Loss=316.320007,Training Ac